<p><font size=6 color="#2E1698"><em><u><center>Data Processing</center></u></em></font></p><br>
<p><font size=3.2 color="#2E1698"><i><u>Introduction:</u></i> explain our data blablabla we are here aiming to manipulate the data that we generated before.</font><p><br>

<font color="#206B50" size = 4><center>**SUMMARY**</center></font><br>
<font size=3.2 color="#2E1698">
<ol>
      <li>Data cleaning</li>
      <li>Provide a list of the 15 most common words</li>
      <li>Provide a list of the 2 pairs of words having the highest co-occurrence frequency</li>
      <li>Build a graphical representation of the most frequent words with their polarity (pos/neg or anger/joy/fear/...)</li>
      <li>Indicate the 3 most frequent representatives words in each category</li>
      <li>Compare the results of the two approaches</li>
    </ol>
</font>

<font size=4 color="#206B50"><center><B>PART I: Data cleaning</B></center></font>
<font color="#2E1698" size = 3.2>Here, we will clean the dataset in order to analyze it.<br><br>
First things first, let's import the csv file</font><br>

In [ ]:
tweets = read.csv("data/debat_primaire_20000.csv", encoding="UTF-8")
print(dim(tweets))
head(tweets, n = 10)

<font color="#2E1698" size = 3.2>So our data frame contains <font color="red">17</font> columns and <font color="red">20 000</font> rows, let's see the 10 firsts rows</font><br>
<font color="#2E1698" size = 3.2>Let's see if all the columns have multiple values, or if some are useless</font>



In [ ]:
head(unique(tweets$favorited))
head(unique(tweets$favoriteCount))
head(unique(tweets$replyToSN))
head(unique(tweets$replyToUID))
head(unique(tweets$id))
head(unique(tweets$isRetweet))
head(unique(tweets$longitude))
head(unique(tweets$latitude))

In [ ]:
length(which(tweets$favorited == "TRUE"))
length(which(tweets$favorited == "FALSE"))

<font  
color="#2E1698" size = 3.2>We can see here that there is no TRUE value for favorited, only FALSE. favorited is useless though.</font>


In [ ]:
length(which(tweets$favoriteCount == 0))
length(which(tweets$favoriteCount != 0))

In [ ]:
(3919/20000)*100

<font color="#2E1698" size = 3.2>The favoriteCount have multiple values, 20% of the are not 0 we better keep this column. It is maybe a significative data</font>


In [ ]:
length(which(tweets$longitude != "NA"))
length(which(tweets$latitude != "NA"))

In [ ]:
(9/20000)*100

<font color="#2E1698" size = 3.2>There is only 9 tweets over 20 000 that contains latitude and longitude, this represents only 0.045% of the tweets, this info can be considered as useless, and we can delete this two columns too.</font>


In [ ]:
length(which(tweets$replyToSN != "NA"))
length(which(tweets$replyToUID != 'NA'))
length(which(tweets$replyToSID != 'NA'))

In [ ]:
(698/20000)*100

<font color="#2E1698" size = 3.2>There is only about 3.5% of the replytoSN and replyToUID data that are not NA, we can delete these two columns as they don't seem to be interesting to study. <br><br>
Let's delete these useless columns!</font>

In [ ]:
tweets <- subset(tweets, select=-c(replyToSN,replyToUID, replyToSID, latitude, longitude, favorited))

In [ ]:
head(tweets, n=10)

<font color="#2E1698" size = 3.2>If we want to use the text, it have to be cleaned first</font>


In [ ]:
clean_text = function(x)
{
    #To convert the text in lowercase
    try.error = function(z)
    {
        y = NA
        try_error = tryCatch(tolower(z), error=function(e) e)
            if (!inherits(try_error, "error"))
                y = tolower(z)
                return(y)
    }
    x = sapply(x, try.error)
            
     #remove all links starting by http
    x = gsub('http\\S+\\s*', '', x)
            
    # replace apostrophes
    x = gsub("'", " ", x)

    # remove punctuation except @, #, _, -
    x = gsub("@", "AAAAAAAAAAA", x)
    x = gsub("#", "BBBBBBBBBBB", x)
    x = gsub("_", "CCCCCCCCCCC", x)
    x = gsub("-", "DDDDDDDDDDD", x)
    x = gsub("[[:punct:]]", " ", x)
    x = gsub("AAAAAAAAAAA", "@", x)
    x = gsub("BBBBBBBBBBB", "#", x)
    x = gsub("CCCCCCCCCCC", "_", x)
    x = gsub("DDDDDDDDDDD", "-", x)
            
    # correcting the spaces after the conserved @
    x = gsub("@ ", "@", x)
            
    # correcting the spaces after the conserved _
    x = gsub("_ ", "_", x)
            
    # correcting the spaces after the conserved -
    x = gsub("- ", "-", x)
    
    # remove numbers/Digits
    x = gsub("[[:digit:]]", "", x)
    
    # remove tabs
    x = gsub("[ |\t]{2,}", " ", x)
            
    # remove blank spaces at the beginning/end
    x = gsub("^ ", "", x)  
    x = gsub(" $", "", x)
    
    
    # As we have already a column indicating if the tweet is a retweet or not 
    # we can remove "RT @xxx" in the tweet header
    x = gsub("rt @\\w+ *", "", x)
    x = gsub('\\b\\w{1,3}\\s','', x)
            
    # remove double spaces
    x = gsub("  ", " ", x)
    x = gsub("  ", " ", x)
    return(x)
}
                             
tweets$text <- clean_text(tweets$text)
head(tweets, n = 20)

<font color="#2E1698" size = 3.2>Let's see which are the most used @xxx and replace them with words. Afterward we will delete all the @xxx that will not be replaced</font>


In [ ]:
col = tweets$text
head(col, n=5)

In [ ]:
at.pattern = "@\\w+ *"
have.at = grep(x = col, pattern = at.pattern)
at.matches = gregexpr(pattern = at.pattern,
                        text = col[have.at])
extracted.at = regmatches(x = col[have.at], m = at.matches)

# most frequent words
mfw = sort(unlist(extracted.at), decreasing=TRUE)
mfw = gsub(" ", "", mfw)
d = sort(table(unlist(mfw)), decreasing=TRUE)
head(d, n = 50)

In [ ]:
top15 = head(d, n = 15)
# barplot
mar.default <- c(12,4,4,4) + 0.1
par(mar = mar.default + c(0, 0, 0, 0))
barplot(top15, border=NA, las=2, main="Top 15 most frequent @twitter_username", cex.main=1)

<font color="#2E1698" size = 3.2>Let's see which are the most used words</font>

In [ ]:
at.pattern = "[a-zA-Z]\\w+ *"
have.at = grep(x = col, pattern = at.pattern)
at.matches = gregexpr(pattern = at.pattern,
                        text = col[have.at])
extracted.at = regmatches(x = col[have.at], m = at.matches)

# most frequent words
mfw = sort(unlist(extracted.at), decreasing=TRUE)
mfw = gsub(" ", "", mfw)
w = sort(table(unlist(mfw)), decreasing=TRUE)
head(d, n = 20)

In [ ]:
top15 = head(w, n = 15)
top15d = sort(top15, decreasing=FALSE) 
# barplot
mar.default <- c(15,10,5,0) + 0.1
par(mar = mar.default + c(0, 0, 0, 0))
barplot(top15d, border=NA, las=2, main="Top 15 most frequent word", cex.main=1, horiz=TRUE)

In [ ]:
y = "#PrimaireLeDébat @ArnaudMontebourGfdp Voilà pourquoi voter pour #Montebourg dimanche ! #Montebourg2017 #PrimaireLeDebat"
x = "#PrimaireLeDébat @ArnaudMontebourGfdp Voilà pourquoi voter pour #Montebourg dimanche ! #Montebourg2017 #PrimaireLeDebat"
y = tolower(y)
x = tolower(x)

In [ ]:
x = gsub('\\S+montebou\\S+', 'montebourg', x)

y
x